In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%pylab inline
import calendar


Populating the interactive namespace from numpy and matplotlib


In [2]:
xl = pd.ExcelFile("Restaurant Data .xlsx")
xl.sheet_names

['Restaurant Data', 'Restaurant Info']

In [3]:
df1 = xl.parse("Restaurant Data")
df2 = xl.parse("Restaurant Info")


In [4]:
df1.head()

,Restaurant Name,Date,Net Sales,POS Checks,Labor Cost,Game Revenue
0,AL - 01,2014-01-01,1687.0,94,609.61,NaN
1,AL - 01,2014-01-02,2182.0,116,577.59,NaN
2,AL - 01,2014-01-03,2772.0,145,717.09,NaN
3,AL - 01,2014-01-04,2253.0,99,705.19,NaN
4,AL - 01,2014-01-05,1390.0,70,626.58,NaN


In [17]:
x= df2[df2['Go Live Date']<'2014-01-01']
x['Restaurant Name'].nunique()

30

In [18]:
len(df1) 

43063

In [19]:
df1.dtypes 

Restaurant Name            object
Date               datetime64[ns]
Net Sales                 float64
POS Checks                  int64
Labor Cost                float64
Game Revenue              float64
dtype: object

In [20]:
df1.shape

(43063, 6)

In [21]:
df1.describe() 

,Net Sales,POS Checks,Labor Cost,Game Revenue
count,43063.000000,43063.000000,43063.000000,22498.000000
mean,3111.583506,140.493974,645.450729,14.364773
std,1450.416221,59.356652,267.430862,4.673675
min,125.000000,10.000000,18.000000,0.000000
25%,2065.000000,98.000000,447.488450,11.000000
50%,2831.000000,131.000000,596.600000,13.666667
75%,3870.000000,172.000000,791.498400,16.916667
max,12092.000000,513.000000,2632.237900,44.333333


In [22]:
df1.isnull().sum()

Restaurant Name        0
Date                   0
Net Sales              0
POS Checks             0
Labor Cost             0
Game Revenue       20565
dtype: int64

In [23]:
len(df2)

105

In [24]:
 
df2.dtypes

Restaurant Name                     object
Go Live Date                datetime64[ns]
Franchise/Owner                     object
State                               object
Presto Devices Installed           float64
dtype: object

In [25]:
df2.shape

(105, 5)

In [26]:
df2.describe() 

,Presto Devices Installed
count,72.000000
mean,45.722222
std,5.567062
min,25.000000
25%,42.000000
50%,45.000000
75%,50.000000
max,55.000000


In [27]:
df2.isnull().sum()

Restaurant Name              0
Go Live Date                34
Franchise/Owner              0
State                        0
Presto Devices Installed    33
dtype: int64

In [28]:
df3 = pd.merge(df1,df2, on='Restaurant Name' )

In [29]:
df3.shape

(43063, 10)

In [30]:
df3['Month'] = pd.to_datetime(df3['Date'], format='%m').dt.month_name().str.slice(stop=3)
df3['Year'] = df3['Date'].dt.year


In [31]:
#model = df3

In [32]:
df3.head(2)

,Restaurant Name,Date,Net Sales,POS Checks,Labor Cost,Game Revenue,Go Live Date,Franchise/Owner,State,Presto Devices Installed,Month,Year
0,AL - 01,2014-01-01,1687.0,94,609.61,NaN,NaT,Southern Group,AL,NaN,Jan,2014
1,AL - 01,2014-01-02,2182.0,116,577.59,NaN,NaT,Southern Group,AL,NaN,Jan,2014


In [33]:
# All the Restaurant that have the Presto devices installed 
df4 = df3[df3['Date'] >= df3['Go Live Date']]

# All the Restaurant that do not have have the Presto devices installed 
#df5 = df3[((df3['Go Live Date'].isnull()) | (df3['Date'] < df3['Go Live Date']))]
df5 = df3[(df3['Date'] < df3['Go Live Date'])]

In [34]:
df4.shape

(22121, 12)

In [35]:
df5.shape

(7568, 12)

In [69]:
df3['Date'].describe()

count                   43063
unique                    452
top       2014-04-02 00:00:00
freq                      102
first     2014-01-01 00:00:00
last      2015-03-30 00:00:00
Name: Date, dtype: object

In [39]:
# Creating a dictionary to see avg net sales for the resturant after presto devices 
after_group = df4.groupby('Restaurant Name')
dict_after={}
for key in after_group.groups.keys():
    total_days = after_group.get_group(key)['Date'].max()- after_group.get_group(key)['Go Live Date']
    #print(total_days)
    days = total_days.dt.days.iloc[0]
    #print(days)
    total_sales = after_group.get_group(key)['Net Sales'].sum()
    dict_after[key] = total_sales/days
    
#dict_after   

In [40]:
# Creating a dictionary to see impact of labor cost after presto devices 
after_group = df4.groupby('Restaurant Name')
dict_laborafter={}
for key in after_group.groups.keys():
    total_days = after_group.get_group(key)['Date'].max()- after_group.get_group(key)['Go Live Date']
    days = total_days.dt.days.iloc[0]
    labor_cost = after_group.get_group(key)['Labor Cost'].sum()
    dict_laborafter[key] = labor_cost/days
    
    

In [41]:
#dict_laborafter

In [42]:
# Creating a dictionary to see avg net sales for the resturant before presto devices 
before_group = df5.groupby('Restaurant Name')
dict_before={}
for key in before_group.groups.keys():
    total_days = before_group.get_group(key)['Go Live Date'] - before_group.get_group(key)['Date'].min()
    days = total_days.dt.days.iloc[0]
    total_sales = before_group.get_group(key)['Net Sales'].sum()
    dict_before[key] = total_sales/days

In [43]:
#dict_before

In [44]:
# Creating a dictionary to see impact of labor cost before presto devices 
before_group = df5.groupby('Restaurant Name')
dict_laborbefore={}
for key in before_group.groups.keys():
    total_days = before_group.get_group(key)['Go Live Date'] - before_group.get_group(key)['Date'].min()
    days = total_days.dt.days.iloc[0]
    labor_cost = before_group.get_group(key)['Labor Cost'].sum()
    dict_laborbefore[key] = labor_cost/days
    
#dict_laborbefore

In [45]:
# Creating a dictionary to see merge  presto devices sales figures for same resturant 
dict_merge = {'Restaurant Name':[],'Avg_sales_before_presto':[],'Avg_sales_after_presto':[]}
for bk in dict_before:
    if bk in dict_after:
        dict_merge['Restaurant Name'].append(bk)
        dict_merge['Avg_sales_before_presto'].append(dict_before[bk])
        dict_merge['Avg_sales_after_presto'].append(dict_after[bk])
#dict_merge

In [46]:
df_before_after = pd.DataFrame.from_dict(dict_merge)
df_before_after.head(5)

,Restaurant Name,Avg_sales_before_presto,Avg_sales_after_presto
0,AL - 02,2856.428135,3077.400000
1,AL - 04,4979.823353,5303.847458
2,AL - 07,2746.289474,2196.630000
3,AZ - 02,1902.639513,1993.291452
4,AZ - 03,1579.282944,1359.331624


In [66]:
df_before_after.isna().sum()

Restaurant Name            0
Avg_sales_before_presto    0
Avg_sales_after_presto     0
dtype: int64

In [47]:
#merge before and after labor costs 
dict_merge3 = {'Restaurant Name':[],'Avg_labor_cost_before_presto':[],'Avg_labor_cost__after_presto':[]}
for bk in dict_laborbefore:
    if bk in dict_laborafter:
        dict_merge3['Restaurant Name'].append(bk)
        dict_merge3['Avg_labor_cost_before_presto'].append(dict_laborbefore[bk])
        dict_merge3['Avg_labor_cost__after_presto'].append(dict_laborafter[bk])
#dict_merge3

In [48]:
df_laborbefore_after = pd.DataFrame.from_dict(dict_merge3)
df_laborbefore_after.head(5)

,Restaurant Name,Avg_labor_cost_before_presto,Avg_labor_cost__after_presto
0,AL - 02,647.371101,681.541040
1,AL - 04,933.773473,974.158136
2,AL - 07,451.024211,332.540833
3,AZ - 02,477.241187,481.422379
4,AZ - 03,353.805806,393.165485


In [49]:
# All the Restaurant Name where the devices were installed before 2013 so that they form a 
#different control group 
df6 = df3[(df3['Go Live Date'] < '2014-01-01')]
df6.head(3)

,Restaurant Name,Date,Net Sales,POS Checks,Labor Cost,Game Revenue,Go Live Date,Franchise/Owner,State,Presto Devices Installed,Month,Year
3155,AL - 08,2014-01-01,1893.0,65,489.46,8.416667,2013-11-18,Southern Group,AL,53.0,Jan,2014
3156,AL - 08,2014-01-02,2513.0,101,570.69,11.416667,2013-11-18,Southern Group,AL,53.0,Jan,2014
3157,AL - 08,2014-01-03,3071.0,118,757.16,12.833333,2013-11-18,Southern Group,AL,53.0,Jan,2014


In [50]:
df6['Restaurant Name'].unique()

array(['AL - 08', 'AZ - 01', 'AZ - 10', 'Ca - 03', 'Ca - 08', 'Ca - 09',
       'Ca - 10', 'Ca - 14', 'Ca - 17', 'Ca - 21', 'FL - 03', 'FL - 04',
       'FL - 07', 'FL - 09', 'FL - 11', 'FL - 12', 'OK- 01', 'OR - 01',
       'OR - 02', 'TX - 15', 'TX - 17', 'TX - 18', 'TX - 19', 'TX - 20',
       'TX - 21', 'TX - 22', 'TX - 25', 'TX - 26', 'TX - 27', 'WA - 06'],
      dtype=object)

In [51]:
# List of resturants that had devices installed before 2014 
devices_before2014 = df6.groupby('Restaurant Name')
dict_devbefore={}
for key in devices_before2014.groups.keys():
    total_days = devices_before2014.get_group(key)['Date'].max() - devices_before2014.get_group(key)['Date'].min()
    days = total_days.days
    #print(days)
    total_sales = devices_before2014.get_group(key)['Net Sales'].sum()
    dict_devbefore[key] = total_sales/days
    
    
#dict_devbefore

In [52]:
dict_merge2 = {'Restaurant Name':[],'Avg_sales_presto_2013':[]}
for bk in dict_devbefore:
    dict_merge2['Restaurant Name'].append(bk)
    dict_merge2['Avg_sales_presto_2013'].append(dict_devbefore[bk])
      
#dict_merge2

In [53]:
df_devbefore_2014 = pd.DataFrame.from_dict(dict_merge2)
df_devbefore_2014.head(3)

,Restaurant Name,Avg_sales_presto_2013
0,AL - 08,2660.183628
1,AZ - 01,4319.136261
2,AZ - 10,3383.152412


In [54]:
df6.shape

(12274, 12)

In [55]:
# All the Restaurant Name where the devices were  not installed at all 

df7 = df3[df3['Go Live Date'].isnull()]
df7.head(2)

,Restaurant Name,Date,Net Sales,POS Checks,Labor Cost,Game Revenue,Go Live Date,Franchise/Owner,State,Presto Devices Installed,Month,Year
0,AL - 01,2014-01-01,1687.0,94,609.61,NaN,NaT,Southern Group,AL,NaN,Jan,2014
1,AL - 01,2014-01-02,2182.0,116,577.59,NaN,NaT,Southern Group,AL,NaN,Jan,2014


In [67]:
df7['Restaurant Name'].unique()

array(['AL - 01', 'AL - 03', 'AL - 05', 'AL - 06', 'AZ - 04', 'AZ - 05',
       'AZ - 06', 'AZ - 08', 'AZ - 11', 'AZ - 14', 'Ca - 04', 'Ca - 05',
       'Ca - 07', 'Ca - 12', 'Ca - 16', 'Ca - 18', 'Ca - 19', 'FL - 02',
       'FL - 05', 'FL - 06', 'FL - 10', 'OK- 03', 'OK- 07', 'OR - 03',
       'TX - 01', 'TX - 02', 'TX - 03', 'TX - 16', 'TX - 23', 'TX - 24',
       'TX - 28', 'WA - 02', 'WA - 03'], dtype=object)

In [56]:
df7.shape

(13374, 12)

In [57]:
# All the Restaurant Name where the devices were  not installed at all 
nodevices = df7.groupby('Restaurant Name')
dict_nodev={}
for key in nodevices.groups.keys():
    total_days = nodevices.get_group(key)['Date'].max() - nodevices.get_group(key)['Date'].min()
    days = total_days.days
    #print(days)
    total_sales = nodevices.get_group(key)['Net Sales'].sum()
    dict_nodev[key] = total_sales/days
    
    
#dict_nodev

In [58]:
nodevices = df7.groupby('Restaurant Name')
dict_labornodev={}
for key in nodevices.groups.keys():
    total_days = nodevices.get_group(key)['Date'].max() - nodevices.get_group(key)['Date'].min()
    days = total_days.days
    #print(days)
    labor_cost = nodevices.get_group(key)['Labor Cost'].sum()
    dict_labornodev[key] = labor_cost/days
    
    
#dict_labornodev

In [59]:
dict_merge1 = {'Restaurant Name':[],'Avg_sales_no_presto':[]}
for bk in dict_nodev:
    dict_merge1['Restaurant Name'].append(bk)
    dict_merge1['Avg_sales_no_presto'].append(dict_nodev[bk])
      
#dict_merge1

In [60]:
dict_merge4 = {'Restaurant Name':[],'Avg_Labor_cost_no_presto':[]}
for bk in dict_labornodev:
    dict_merge4['Restaurant Name'].append(bk)
    dict_merge4['Avg_Labor_cost_no_presto'].append(dict_labornodev[bk])
      
#dict_merge4

In [61]:
df_nopresto = pd.DataFrame.from_dict(dict_merge1)
df_nopresto.head(3)

,Restaurant Name,Avg_sales_no_presto
0,AL - 01,2057.159292
1,AL - 03,5582.415929
2,AL - 05,3775.234513


In [62]:
df_nopresto_laborcost = pd.DataFrame.from_dict(dict_merge4)
df_nopresto_laborcost.head(3)

,Restaurant Name,Avg_Labor_cost_no_presto
0,AL - 01,522.350553
1,AL - 03,1025.260088
2,AL - 05,709.694491


In [63]:
#Merge df3 dataset with the Total Sales when never had the device
df9 = pd.merge(df3,df_before_after, on='Restaurant Name',how='left' )
df9.head(3)

,Restaurant Name,Date,Net Sales,POS Checks,Labor Cost,Game Revenue,Go Live Date,Franchise/Owner,State,Presto Devices Installed,Month,Year,Avg_sales_before_presto,Avg_sales_after_presto
0,AL - 01,2014-01-01,1687.0,94,609.61,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN
1,AL - 01,2014-01-02,2182.0,116,577.59,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN
2,AL - 01,2014-01-03,2772.0,145,717.09,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN


In [64]:
df10 = pd.merge(df9,df_nopresto, on='Restaurant Name',how='left' )

In [65]:
df10.head(3)

,Restaurant Name,Date,Net Sales,POS Checks,Labor Cost,Game Revenue,Go Live Date,Franchise/Owner,State,Presto Devices Installed,Month,Year,Avg_sales_before_presto,Avg_sales_after_presto,Avg_sales_no_presto
0,AL - 01,2014-01-01,1687.0,94,609.61,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN,2057.159292
1,AL - 01,2014-01-02,2182.0,116,577.59,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN,2057.159292
2,AL - 01,2014-01-03,2772.0,145,717.09,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN,2057.159292


In [1015]:
df10['Difference']= ((df10['Net Sales']+ df10.fillna(0)['Game Revenue']) - (df10['Labor Cost']))
df10.head(2)

,Restaurant Name,Date,Net Sales,POS Checks,Labor Cost,Game Revenue,Go Live Date,Franchise/Owner,State,Presto Devices Installed,Month,Year,Avg_sales_before_presto,Avg_sales_after_presto,Avg_sales_no_presto,Difference
0,AL - 01,2014-01-01,1687.0,94,609.61,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN,2057.159292,1077.39
1,AL - 01,2014-01-02,2182.0,116,577.59,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN,2057.159292,1604.41


In [1016]:
df10.shape

(43063, 16)

In [1017]:
df11 = pd.merge(df10,df_devbefore_2014, on='Restaurant Name',how='left' )
df11.head(3)

,Restaurant Name,Date,Net Sales,POS Checks,Labor Cost,Game Revenue,Go Live Date,Franchise/Owner,State,Presto Devices Installed,Month,Year,Avg_sales_before_presto,Avg_sales_after_presto,Avg_sales_no_presto,Difference,Avg_sales_presto_2013
0,AL - 01,2014-01-01,1687.0,94,609.61,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN,2057.159292,1077.39,NaN
1,AL - 01,2014-01-02,2182.0,116,577.59,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN,2057.159292,1604.41,NaN
2,AL - 01,2014-01-03,2772.0,145,717.09,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN,2057.159292,2054.91,NaN


In [1018]:
df11.shape

(43063, 17)

In [1019]:
df12 = pd.merge(df11,df_laborbefore_after, on='Restaurant Name',how='left' )
df12.head(3)

,Restaurant Name,Date,Net Sales,POS Checks,Labor Cost,Game Revenue,Go Live Date,Franchise/Owner,State,Presto Devices Installed,Month,Year,Avg_sales_before_presto,Avg_sales_after_presto,Avg_sales_no_presto,Difference,Avg_sales_presto_2013,Avg_labor_cost_before_presto,Avg_labor_cost__after_presto
0,AL - 01,2014-01-01,1687.0,94,609.61,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN,2057.159292,1077.39,NaN,NaN,NaN
1,AL - 01,2014-01-02,2182.0,116,577.59,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN,2057.159292,1604.41,NaN,NaN,NaN
2,AL - 01,2014-01-03,2772.0,145,717.09,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN,2057.159292,2054.91,NaN,NaN,NaN


In [1020]:

df13 = pd.merge(df12,df_nopresto_laborcost, on='Restaurant Name',how='left' )
df13.head(3)

,Restaurant Name,Date,Net Sales,POS Checks,Labor Cost,Game Revenue,Go Live Date,Franchise/Owner,State,Presto Devices Installed,Month,Year,Avg_sales_before_presto,Avg_sales_after_presto,Avg_sales_no_presto,Difference,Avg_sales_presto_2013,Avg_labor_cost_before_presto,Avg_labor_cost__after_presto,Avg_Labor_cost_no_presto
0,AL - 01,2014-01-01,1687.0,94,609.61,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN,2057.159292,1077.39,NaN,NaN,NaN,522.350553
1,AL - 01,2014-01-02,2182.0,116,577.59,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN,2057.159292,1604.41,NaN,NaN,NaN,522.350553
2,AL - 01,2014-01-03,2772.0,145,717.09,NaN,NaT,Southern Group,AL,NaN,Jan,2014,NaN,NaN,2057.159292,2054.91,NaN,NaN,NaN,522.350553


In [625]:
df13.shape

(43063, 20)

In [1022]:
df13['Sale Month'] = pd.to_datetime(df13['Date'], format='%m').dt.month_name().str.slice(stop=3)
df13['Sale Year'] = df13['Date'].dt.year

In [1023]:
df13['Go Live Month'] = pd.to_datetime(df13['Go Live Date'], format='%m').dt.month_name().str.slice(stop=3)
#df11['Go Live Year'] = df11['Go Live Date'].dt.year

In [1069]:
df13.to_csv('Cleaned1.csv', sep='\t', encoding='utf-8')

In [1026]:
df4_monthly = df4.groupby(['Restaurant Name','Month','Year'])['Net Sales'].sum()
df4_monthly = df4_monthly.reset_index()
df4_monthly.head(2)

,Restaurant Name,Month,Year,Net Sales
0,AL - 02,Dec,2014,77885.0
1,AL - 02,Feb,2015,102101.0


In [1027]:
df4_labor_monthly = df4.groupby(['Restaurant Name','Month','Year'])['Labor Cost'].sum()
df4_labor_monthly = df4_labor_monthly.reset_index()
df4_labor_monthly.head(2)

,Restaurant Name,Month,Year,Labor Cost
0,AL - 02,Dec,2014,18281.41
1,AL - 02,Feb,2015,21210.65


In [1029]:
#df5 #before

In [1030]:
df5_monthly = df5.groupby(['Restaurant Name','Month','Year'])['Net Sales'].sum()
df5_monthly = df5_monthly.reset_index()
#df5_monthly

In [1032]:
df5_labor_monthly = df5.groupby(['Restaurant Name','Month','Year'])['Labor Cost'].sum()
df5_labor_monthly = df5_labor_monthly.reset_index()
df5_labor_monthly.head(2)

,Restaurant Name,Month,Year,Labor Cost
0,AL - 02,Apr,2014,17771.45
1,AL - 02,Aug,2014,20844.44


In [667]:
#df_common_months['Restaurant Name'].nunique()

In [1033]:
df_common_months = pd.merge(df4_monthly,df5_monthly,on=['Restaurant Name','Month'])[['Restaurant Name','Month','Net Sales_x','Net Sales_y']]
df_common_months.head(5)

,Restaurant Name,Month,Net Sales_x,Net Sales_y
0,AL - 02,Feb,102101.0,83937.0
1,AL - 02,Jan,93814.0,84910.0
2,AL - 02,Mar,95745.0,90630.0
3,AL - 02,Nov,15130.0,63441.0
4,AL - 04,Feb,159181.0,139019.0


In [1099]:
df_common_months['Difference']=((df_common_months['Net Sales_x']-df_common_months['Net Sales_y'])/(df_common_months['Net Sales_x']))*100

In [1100]:
df_common_months.head()

,Restaurant Name,Month,Net Sales_x,Net Sales_y,Difference
0,AL - 02,Feb,102101.0,83937.0,17.790227
1,AL - 02,Jan,93814.0,84910.0,9.491121
2,AL - 02,Mar,95745.0,90630.0,5.342316
3,AL - 02,Nov,15130.0,63441.0,-319.306015
4,AL - 04,Feb,159181.0,139019.0,12.666085


In [1036]:
df_common_months_labor = pd.merge(df4_labor_monthly,df5_labor_monthly,on=['Restaurant Name','Month'])[['Restaurant Name','Month','Labor Cost_x','Labor Cost_y']]
df_common_months_labor.head(2)

,Restaurant Name,Month,Labor Cost_x,Labor Cost_y
0,AL - 02,Feb,21210.65,18040.46
1,AL - 02,Jan,20841.80,19322.49


In [1092]:
df_common_months.head(2)

,Restaurant Name,Month,Net Sales_x,Net Sales_y
0,AL - 02,Feb,102101.0,83937.0
1,AL - 02,Jan,93814.0,84910.0


In [1093]:
df7_monthly = df7.groupby(['Restaurant Name','Franchise/Owner','Month','Year'])['Net Sales'].sum()
df7_monthly = df7_monthly.reset_index()
df7_monthly.head(7)

,Restaurant Name,Franchise/Owner,Month,Year,Net Sales
0,AL - 01,Southern Group,Apr,2014,61595.0
1,AL - 01,Southern Group,Aug,2014,68209.0
2,AL - 01,Southern Group,Dec,2014,58466.0
3,AL - 01,Southern Group,Feb,2014,56308.0
4,AL - 01,Southern Group,Feb,2015,68112.0
5,AL - 01,Southern Group,Jan,2014,62338.0
6,AL - 01,Southern Group,Jan,2015,67680.0


In [1094]:
df_common_months1 = pd.merge(df_common_months,df7_monthly,on=['Month'])[['Month','Net Sales_x','Net Sales_y','Net Sales']]
df_common_months1.head(5)

,Month,Net Sales_x,Net Sales_y,Net Sales
0,Feb,102101.0,83937.0,56308.0
1,Feb,102101.0,83937.0,68112.0
2,Feb,102101.0,83937.0,150664.0
3,Feb,102101.0,83937.0,193727.0
4,Feb,102101.0,83937.0,114023.0


In [1095]:
df_common_months.to_csv('df_common_months.csv', sep='\t', encoding='utf-8')

In [1096]:
df_common_months_labor.to_csv('df_common_months1.csv', sep='\t', encoding='utf-8')